In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1643708810590_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# importing required libraries
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType
from pyspark.sql.functions import *
from pyspark.sql.functions import concat, lit, col
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as sf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Using "StructType" to manually create datatype for specified columns
fileSchema = StructType([StructField('year', IntegerType(),True),
                        StructField('month', StringType(),True),
                        StructField('day', IntegerType(),True),
                        StructField('weekday', StringType(),True),
                        StructField('hour', IntegerType(),True),
                        StructField('atm_status', StringType(),True),
                        StructField('atm_id', StringType(),True),
                        StructField('atm_manufacturer', StringType(),True),
                        StructField('atm_location', StringType(),True),
                        StructField('atm_streetname', StringType(),True),
                        StructField('atm_street_number', IntegerType(),True),
                        StructField('atm_zipcode', IntegerType(),True),
                        StructField('atm_lat', DoubleType(),True),
                        StructField('atm_lon', DoubleType(),True),
                        StructField('currency', StringType(),True),
                        StructField('card_type', StringType(),True),
                        StructField('transaction_amount', IntegerType(),True), 
                        StructField('service', StringType(),True),
                        StructField('message_code', StringType(),True),
                        StructField('message_text', StringType(),True),
                        StructField('weather_lat', DoubleType(),True),
                        StructField('weather_lon', DoubleType(),True),
                        StructField('weather_city_id', IntegerType(),True),
                        StructField('weather_city_name', StringType(),True), 
                        StructField('temp', DoubleType(),True),
                        StructField('pressure', IntegerType(),True),
                        StructField('humidity', IntegerType(),True),
                        StructField('wind_speed', IntegerType(),True),
                        StructField('wind_deg', IntegerType(),True),
                        StructField('rain_3h', DoubleType(),True),
                        StructField('clouds_all', IntegerType(),True),
                        StructField('weather_id', IntegerType(),True),
                        StructField('weather_main', StringType(),True),
                        StructField('weather_description', StringType(),True),
                        ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Reading data
ATM_data = spark.read.csv("/user/root/SRC_ATM_TRANS/",header=False,schema = fileSchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#Getting count of mainfile
ATM_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [6]:
ATM_data.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468300

### Creating DIM_LOC

In [7]:
DIM_LOC_TEMP  = ATM_data.select("atm_location", "atm_streetname","atm_street_number","atm_zipcode", "atm_lat", "atm_lon").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
DIM_LOC_temp = DIM_LOC_TEMP.dropDuplicates((["atm_location", "atm_streetname","atm_street_number","atm_zipcode", "atm_lat", "atm_lon"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
DIM_LOC_temp.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [10]:
DIM_LOC_temp = DIM_LOC_temp.withColumn("location_id",row_number().over(Window.orderBy(monotonically_increasing_id())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
DIM_LOC_temp.orderBy('location_id', ascending=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[atm_location: string, atm_streetname: string, atm_street_number: int, atm_zipcode: int, atm_lat: double, atm_lon: double, location_id: int]

In [12]:
DIM_LOC = DIM_LOC_temp.select(col('location_id'),col('atm_location').alias("location"),col('atm_streetname').alias('streetname'),col('atm_street_number').alias('street_number'),col('atm_zipcode').alias('zipcode'),col('atm_lat').alias('lat'),col('atm_lon').alias('lon'))
DIM_LOC.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+-------------------+-------------+-------+------+------+
|location_id|        location|         streetname|street_number|zipcode|   lat|   lon|
+-----------+----------------+-------------------+-------------+-------+------+------+
|          1|         Kolding|           Vejlevej|          135|   6000|55.505| 9.457|
|          2|  Skelagervej 15|        Skelagervej|           15|   9000|57.023| 9.891|
|          3|Intern HolbÃƒÂ¦k|        Slotsvolden|            7|   4300|55.718|11.704|
|          4|          Odense|       FÃƒÂ¦lledvej|            3|   5000|55.394| 10.37|
|          5|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|           12|   7430|56.139| 9.154|
|          6|         Randers|       ÃƒËœstervold|           16|   8900|56.462|10.038|
|          7|         Esbjerg|       Strandbygade|           20|   6700|55.468|  8.44|
|          8|          Lyngby|        Jernbanevej|            6|   2800|55.772|  12.5|
|          9|           Skive|           Ad

### Creating DIM_ATM

In [13]:
DIM_ATM_TEMP = ATM_data.select("atm_id", "atm_lat","atm_lon","atm_manufacturer")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
DIM_ATM_TEMP.registerTempTable("atm")
DIM_LOC_temp.registerTempTable("loc")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
DIM_ATM_1= spark.sql("select atm.atm_id,atm.atm_manufacturer,loc.*from atm left join loc on atm.atm_lat==loc.atm_lat and atm.atm_lon==loc.atm_lon").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
DIM_ATM_2 =  DIM_ATM_1.withColumn("atm_number",row_number().over(Window.orderBy(monotonically_increasing_id())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
DIM_ATM_2.orderBy('atm_id', ascending=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[atm_id: string, atm_manufacturer: string, atm_location: string, atm_streetname: string, atm_street_number: int, atm_zipcode: int, atm_lat: double, atm_lon: double, location_id: int, atm_number: int]

In [18]:
DIM_ATM = DIM_ATM_2.select("atm.atm_id", "atm_number","atm_manufacturer","location_id")
DIM_ATM.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+-----------+
|atm_id|atm_number|atm_manufacturer|location_id|
+------+----------+----------------+-----------+
|   102|         1|             NCR|         50|
|    10|         2|             NCR|         96|
|    50|         3|             NCR|         32|
|    89|         4|             NCR|         59|
|    73|         5|             NCR|         85|
|   101|         6|             NCR|         72|
|    92|         7|             NCR|         51|
|    20|         8|             NCR|         37|
|    30|         9|             NCR|         58|
|    24|        10|             NCR|         13|
|    46|        11| Diebold Nixdorf|         20|
|    63|        12|             NCR|         84|
|    66|        13|             NCR|         21|
|    12|        14|             NCR|         70|
|    86|        15|             NCR|        105|
|    16|        16|             NCR|         67|
|    82|        17|             NCR|         54|
|    41|        18| 

In [19]:
DIM_ATM = DIM_ATM.dropDuplicates((["atm_id"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
DIM_ATM.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

### creating DIM_DATE

In [21]:
dim_date = ATM_data.select('year','month','day','hour','weekday').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
dim_date.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+---+----+--------+
|year| month|day|hour| weekday|
+----+------+---+----+--------+
|2017|August|  1|  11| Tuesday|
|2017|August|  3|  21|Thursday|
|2017|  July| 29|  17|Saturday|
+----+------+---+----+--------+
only showing top 3 rows

In [23]:
DIM_DATE = dim_date.withColumn("full_date_time",from_unixtime(unix_timestamp(concat(dim_date.year.cast(StringType()),dim_date.month.cast(StringType()),lpad(dim_date.day.cast(StringType()),2,'0'),lpad(dim_date.hour.cast(StringType()),2,'0')),'yyyyMMMMddHH'),'YYYY-MM-dd HH:mm:SS'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
DIM_DATE = DIM_DATE.withColumn("date_id",F.row_number().over(Window.partitionBy().orderBy(DIM_DATE["year"])))
DIM_DATE.show(40)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+---+----+---------+-------------------+-------+
|year|   month|day|hour|  weekday|     full_date_time|date_id|
+----+--------+---+----+---------+-------------------+-------+
|2017| January|  5|  21| Thursday|2017-01-05 21:00:00|      1|
|2017| January| 22|  15|   Sunday|2017-01-22 15:00:00|      2|
|2017| January| 23|  18|   Monday|2017-01-23 18:00:00|      3|
|2017| January|  4|   4|Wednesday|2017-01-04 04:00:00|      4|
|2017|February|  8|  13|Wednesday|2017-02-08 13:00:00|      5|
|2017|February| 18|  21| Saturday|2017-02-18 21:00:00|      6|
|2017| January| 24|  14|  Tuesday|2017-01-24 14:00:00|      7|
|2017|February|  6|  15|   Monday|2017-02-06 15:00:00|      8|
|2017| January| 27|   2|   Friday|2017-01-27 02:00:00|      9|
|2017| January| 14|   5| Saturday|2017-01-14 05:00:00|     10|
|2017| January| 16|  19|   Monday|2017-01-16 19:00:00|     11|
|2017| January| 17|  19|  Tuesday|2017-01-17 19:00:00|     12|
|2017|February| 13|  14|   Monday|2017-02-13 14:00:00| 

In [25]:
DIM_DATE=DIM_DATE.select("date_id","full_date_time","year","month","day","hour","weekday")
DIM_DATE.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+------+---+----+---------+
|date_id|     full_date_time|year| month|day|hour|  weekday|
+-------+-------------------+----+------+---+----+---------+
|      1|2017-08-01 11:00:00|2017|August|  1|  11|  Tuesday|
|      2|2017-08-03 21:00:00|2017|August|  3|  21| Thursday|
|      3|2017-07-29 17:00:00|2017|  July| 29|  17| Saturday|
|      4|2017-08-03 15:00:00|2017|August|  3|  15| Thursday|
|      5|2017-08-13 02:00:00|2017|August| 13|   2|   Sunday|
|      6|2017-07-06 14:00:00|2017|  July|  6|  14| Thursday|
|      7|2017-07-13 16:00:00|2017|  July| 13|  16| Thursday|
|      8|2017-07-30 13:00:00|2017|  July| 30|  13|   Sunday|
|      9|2017-08-03 05:00:00|2017|August|  3|   5| Thursday|
|     10|2017-08-12 22:00:00|2017|August| 12|  22| Saturday|
|     11|2017-07-14 02:00:00|2017|  July| 14|   2|   Friday|
|     12|2017-08-02 22:00:00|2017|August|  2|  22|Wednesday|
|     13|2017-08-10 18:00:00|2017|August| 10|  18| Thursday|
|     14|2017-07-03 10:0

### Creating DIM_CARD_TYPE

In [26]:
# Identifying CARD TYPE DIM From Source File.
TEMP_DIM_CARD= ATM_data.select('card_type')
DIM_CARD_TYPE = TEMP_DIM_CARD.dropDuplicates((['card_type']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
DIM_CARD_TYPE = DIM_CARD_TYPE.withColumn("card_type_id",row_number().over(Window.orderBy(monotonically_increasing_id())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
DIM_CARD_TYPE =DIM_CARD_TYPE.select("card_type_id","card_type")
DIM_CARD_TYPE.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|           1|Visa Dankort - on-us|
|           2|  Mastercard - on-us|
|           3|         HÃƒÂ¦vekort|
|           4|            VisaPlus|
|           5|     Dankort - on-us|
|           6|        Visa Dankort|
|           7| HÃƒÂ¦vekort - on-us|
|           8|              CIRRUS|
|           9|                VISA|
|          10|             Maestro|
|          11|          MasterCard|
|          12|             Dankort|
+------------+--------------------+

In [29]:
file1 = ATM_data.join(DIM_ATM_1,on=['atm_id','atm_manufacturer','atm_location','atm_streetname','atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon'],how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
file1.filter(file1.location_id.isNotNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [31]:
file1 = file1.withColumnRenamed('location_id','tlocation_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
file2 = file1.join(DIM_ATM_2,on=['atm_id','atm_manufacturer','atm_location','atm_streetname','atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon'],how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
file2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [34]:
file2.filter(file2.atm_number.isNotNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [35]:
file2 = file2.drop('tlocation_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
file3 = file2.withColumnRenamed('card_type','card_types')
ATM_data_1 = file3.join(DIM_CARD_TYPE, file3.card_types == DIM_CARD_TYPE.card_type,how='LEFT')  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
ATM_data_1 .filter(ATM_data_1.card_type_id.isNotNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [38]:
ATM_data_1 = ATM_data_1.withColumn("full_date_time",from_unixtime(unix_timestamp(concat(dim_date.year.cast(StringType()),dim_date.month.cast(StringType()),lpad(dim_date.day.cast(StringType()),2,'0'),lpad(dim_date.hour.cast(StringType()),2,'0')),'yyyyMMMMddHH'),'YYYY-MM-dd HH:mm:SS'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
ATM_data_1 = ATM_data_1.withColumnRenamed('year','ryear')
ATM_data_1 = ATM_data_1.withColumnRenamed('month','rmonth')
ATM_data_1 = ATM_data_1.withColumnRenamed('day','rday')
ATM_data_1 = ATM_data_1.withColumnRenamed('weekday','rweekday')
ATM_data_1 = ATM_data_1.withColumnRenamed('hour','rhour')
ATM_data_1 = ATM_data_1.withColumnRenamed('full_date_time','rfull_date_time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
ATM_data_1 = ATM_data_1.join(DIM_DATE, ATM_data_1.rfull_date_time == DIM_DATE.full_date_time,how='LEFT') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
ATM_data_1 = ATM_data_1.drop('year','month','day','weekday','hour','full_date_time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
ATM_data_1 = ATM_data_1.withColumnRenamed('ryear','year')
ATM_data_1 = ATM_data_1.withColumnRenamed('rmonth','month')
ATM_data_1 = ATM_data_1.withColumnRenamed('rday','day')
ATM_data_1 = ATM_data_1.withColumnRenamed('rweekday','weekday')
ATM_data_1 = ATM_data_1.withColumnRenamed('rhour','hour')
ATM_data_1 = ATM_data_1.withColumnRenamed('rfull_date_time','full_date_time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
ATM_data_1 = ATM_data_1.withColumn("trans_id",row_number().over(Window.orderBy(monotonically_increasing_id())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Creating FACT_ATM_TRANS

In [44]:
FACT_ATM_TRANS = ATM_data_1.select("trans_id","atm_id", "location_id","date_id","card_type_id","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [46]:
FACT_ATM_TRANS.select("atm_id").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [47]:
FACT_ATM_TRANS.select("location_id").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [48]:
FACT_ATM_TRANS.select("date_id").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [49]:
FACT_ATM_TRANS.select("card_type_id").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [50]:
FACT_ATM_TRANS =FACT_ATM_TRANS.withColumnRenamed('location_id','weather_loc_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
FACT_ATM_TRANS = FACT_ATM_TRANS.select("trans_id","atm_id","weather_loc_id","date_id","card_type_id","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")
FACT_ATM_TRANS.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|       1|    51|            30|   8576|           1|    Active|     DKK|Withdrawal|              3562|        null|        null|  1.105|        92|       500|        Rain|         light rain|
|       2|    51|            30|   1195|           2|    Active|     DKK|Withdrawal|              8441|        null|        null|    0.0|        92|       804|      Clouds|    overcast clouds|
|       3|    51|            30|   

### Copying into s3 bucket

In [52]:
DIM_ATM.write.csv("s3://nikhil-etl/DIM_ATM.csv",mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
DIM_LOC.write.csv("s3://nikhil-etl/DIM_LOCATION.csv",mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
DIM_DATE.write.csv("s3://nikhil-etl/DIM_DATE.csv",mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
DIM_CARD_TYPE.write.csv("s3://nikhil-etl/DIM_CARD_TYPE.csv",mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
FACT_ATM_TRANS.write.csv("s3://nikhil-etl/FACT_ATM_TRANS.csv",mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…